# Julia 深度學習：卷積神經網路模型簡介

本範例有可選用套件 CuArrays，請在執行以下範例前先安裝。

```
] add CuArrays
```

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

## 載入資料

In [2]:
train_X, train_y = MNIST.traindata(Float32)
test_X, test_y = MNIST.testdata(Float32)

(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [3]:
train_X = reshape(train_X, 28, 28, 1, :)
test_X = reshape(test_X, 28, 28, 1, :)
train_y = onehotbatch(train_y, 0:9)
test_y = onehotbatch(test_y, 0:9)

10×10000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  0  1  0  0  0  0  0  0  1  0  0  …  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  1  0  0  1  0  0  0  0  0  0  0     0  0  0  0  0  0  1  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  1  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0  …  1  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  1  0     0  1  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  1  0  0  1     0  0  0  0  1  0  0  0  0  0  0  0

In [4]:
batchsize = 1024
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)
test = DataLoader(test_X, test_y, batchsize=batchsize)

DataLoader((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]), 1024, 10000, true, 10000, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000], false)

## CNN 模型

In [5]:
model = Chain(
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    flatten,
    Dense(288, 10),
    softmax)

Chain(Conv((3, 3), 1=>16, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 16=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(288, 10), softmax)

## 使用 CUDA

In [ ]:
using CuArrays
model = model |> gpu
train_X = train_X |> gpu
train_y = train_y |> gpu
test_X = test_X |> gpu
test_y = test_y |> gpu

## 損失函數

In [6]:
loss(x, y) = logitcrossentropy(model(x), y)

loss (generic function with 1 method)

## Callback 函式

In [7]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l/length(test)
end

test_loss (generic function with 1 method)

In [8]:
evalcb() = @show(test_loss())

evalcb (generic function with 1 method)

## 模型訓練

In [9]:
epochs = 20
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb=throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\a504082002\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2978976f0
test_loss() = 2.2789884f0
test_loss() = 2.1853604f0
test_loss() = 2.0578403f0
test_loss() = 1.9240929f0
test_loss() = 1.7921705f0
test_loss() = 1.7451874f0
test_loss() = 1.7059281f0
test_loss() = 1.6573483f0
test_loss() = 1.6132984f0
test_loss() = 1.5923207f0
test_loss() = 1.5754845f0
test_loss() = 1.5417564f0
test_loss() = 1.5450425f0
test_loss() = 1.5310905f0
test_loss() = 1.5230321f0
test_loss() = 1.5199292f0
test_loss() = 1.5120493f0
test_loss() = 1.5137494f0


┌ Info: Epoch 2
└ @ Main C:\Users\a504082002\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 1.7913816f0
test_loss() = 1.6230695f0
test_loss() = 1.5913196f0
test_loss() = 1.5229938f0
test_loss() = 1.5260439f0
test_loss() = 1.5142108f0
test_loss() = 1.5061481f0
test_loss() = 1.508243f0
test_loss() = 1.5036813f0
test_loss() = 1.4985608f0
test_loss() = 1.4992458f0
test_loss() = 1.4971577f0
test_loss() = 1.4953554f0
test_loss() = 1.4960064f0


┌ Info: Epoch 3
└ @ Main C:\Users\a504082002\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

test_loss() = 1.4781134f0
test_loss() = 1.4787269f0
test_loss() = 1.4742867f0
test_loss() = 1.4734752f0
test_loss() = 1.4720042f0
test_loss() = 1.4712938f0
test_loss() = 1.4738601f0
test_loss() = 1.4712892f0
test_loss() = 1.4710546f0
test_loss() = 1.47231f0
test_loss() = 1.4713365f0


┌ Info: Epoch 19
└ @ Main C:\Users\a504082002\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 1.4725168f0
test_loss() = 1.4741551f0
test_loss() = 1.472223f0
test_loss() = 1.4724032f0
test_loss() = 1.4727005f0
test_loss() = 1.4727395f0
test_loss() = 1.472049f0
test_loss() = 1.4701542f0
test_loss() = 1.472879f0
test_loss() = 1.4708837f0
test_loss() = 1.4722389f0


┌ Info: Epoch 20
└ @ Main C:\Users\a504082002\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 1.4711142f0
test_loss() = 1.471599f0
test_loss() = 1.4719744f0
test_loss() = 1.4710052f0
test_loss() = 1.4710711f0
test_loss() = 1.4725395f0
test_loss() = 1.4716108f0
test_loss() = 1.4729892f0
test_loss() = 1.474802f0
test_loss() = 1.4719163f0
test_loss() = 1.4706767f0
test_loss() = 1.4706908f0


## 模型評估

In [10]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [11]:
accuracy(test_X, test_y)

0.9904